In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

开始使用Vertex AI管道机器管理

<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/get_started_with_machine_management.ipynb">
  <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
</a>

<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/get_started_with_machine_management.ipynb">
  <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
  在GitHub上查看
</a>

<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/get_started_with_machine_management.ipynb">
  <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
  在Vertex AI工作台中打开
</a>

## 概览

本教程演示了如何在“Vertex AI Pipelines”中作为组件训练时管理机器资源。

### 目标

在这个教程中，您将学习如何将一个自包含的自定义训练组件转换为`Vertex AI CustomJob`，实现以下功能：

    - 可跟踪训练作业和生成的工件。
    - 设置机器资源，如机器类型、CPU/GPU、内存、磁盘等。

此教程使用以下谷歌云ML服务：

- `Vertex AI Pipelines`

本教程中执行的步骤包括：

- 创建一个具有自包含训练作业的自定义组件。
- 使用组件级别设置执行管道以设置机器资源。
- 将自包含训练组件转换为`Vertex AI CustomJob`。
- 使用CustomJob级别设置执行管道以设置机器资源。

数据集

该数据集是MNIST数据集。该数据集由28x28像素的灰度图像组成，包含数字0至9。

### 成本

本教程使用了 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

安装所需的软件包以执行此笔记本。

In [ ]:
import os

! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-pipeline-components --quiet
! pip3 install --upgrade kfp --quiet
! pip3 install --upgrade tensorflow==2.7 --quiet

只有Colab：取消注释下面的单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行`gcloud config list`。
* 运行`gcloud projects list`。
* 查看支持页面：[找到项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 地区

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

1. 顶点人工智能工作台
* 无需操作，因为您已经通过身份验证。

本地的JupyterLab实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐户或其他
* 查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上授予您的服务帐户云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

#### 服务账号

**如果您不知道您的服务账号**，请尝试使用`gcloud`命令来获取您的服务账号，执行下面的第二个单元格。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        # print("shell_output=", shell_output)
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI Pipelines设置服务账号访问权限

运行以下命令来授予您的服务账号权限，在您上一步创建的存储桶中读取和写入管道文件 -- 您只需要在每个服务账号上运行一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库

In [ ]:
import json

import tensorflow as tf
from google.cloud import aiplatform
from google_cloud_pipeline_components.v1.custom_job import \
    create_custom_training_job_from_component
from kfp import compiler, dsl
from kfp.dsl import component

初始化Python的Vertex AI SDK，为您的项目和相应的存储桶进行初始化。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持 GPU 的容器镜像以及分配给虚拟机实例的 GPU 数量。例如，要使用一个 GPU 容器镜像，并为每个 VM 分配 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则指定`(None, None)`以使用在 CPU 上运行的容器镜像。

了解更多关于[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：在 TF 2.3 之前的 GPU 支持版本将无法加载本教程中的自定义模型。这是一个已知问题，在 TF 2.3 中已得到修复。这是由生成在服务函数中的静态图操作引起的。如果在自己的定制模型中遇到此问题，请使用带有 GPU 支持的 TF 2.3 容器镜像。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建Docker容器映像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.5".replace(".", "-")
TRAIN_VERSION = "tf-gpu.{}".format(TF)
DEPLOY_VERSION = "tf2-gpu.{}".format(TF)


TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU的内存为3.75GB。
     - `n1-highmem`: 每个vCPU的内存为6.5GB。
     - `n1-highcpu`: 每个vCPU的内存为0.9GB。
 - `vCPU数目`: \[2, 4, 8, 16, 32, 64, 96 \]

*注意: 以下机型不支持用于训练:*

 - `standard`: 2个vCPU
 - `highcpu`: 2、4和8个vCPU

*注意: 您也可以使用n2和e2机型进行训练和部署，但它们不支持GPU。*

In [ ]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

DEPLOY_COMPUTE = "n1-standard-4"
print("Deploy machine type", DEPLOY_COMPUTE)

## 创建一个自包含的自定义训练组件

首先，您可以创建一个包含整个训练步骤的组件。该组件使用 TensorFlow 框架训练一个简单的 MNIST 模型。训练完全包含在组件中：

- 获取并预处理数据。
- 获取/构建模型。
- 训练模型。
- 保存模型。

该组件接受以下参数：

- `model_dir`：保存训练模型构件的 Cloud Storage 位置。
- `epochs`：训练模型的时代数。

In [ ]:
@component(
    base_image=TRAIN_IMAGE,
    packages_to_install=["tensorflow"],
)
def self_contained_training_component(
    model_dir: str,
    epochs: int,
) -> str:
    import numpy as np

    def get_data():
        from tensorflow.keras.datasets import mnist

        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train = (x_train / 255.0).astype(np.float32)
        x_test = (x_test / 255.0).astype(np.float32)

        return (x_train, y_train, x_test, y_test)

    def get_model():
        from tensorflow.keras import Sequential
        from tensorflow.keras.layers import Dense, Flatten

        model = Sequential(
            [
                Flatten(input_shape=(28, 28, 1)),
                Dense(128, activation="relu"),
                Dense(256, activation="relu"),
                Dense(128, activation="relu"),
                Dense(10, activation="softmax"),
            ]
        )

        model.compile(
            optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["acc"]
        )

        return model

    def train_model(x_train, y_train, model, epochs):
        history = model.fit(x_train, y_train, epochs=epochs)
        return history

    (x_train, y_train, _, _) = get_data()
    model = get_model()
    train_model(x_train, y_train, model, epochs)

    model.save(model_dir)
    return model_dir


compiler.Compiler().compile(self_contained_training_component, "demo_componet.yaml")

## 创建自包含的训练管道

接下来，您需要创建用于训练该组件的管道，包括以下步骤：

- *训练模型*。对于这个组件，您需要设置以下组件级别的资源：
    - `cpu_limit`：容器的虚拟机实例的CPU数量。
    - `memory_limit`：容器的虚拟机实例的内存量。
    - `node_selector_constraint`：容器的虚拟机实例的GPU类型。
    - `gpu_limit`：容器的虚拟机实例的GPU数量。
- *将模型工件导入到模型容器工件中*。
- *将容器工件上传到`Vertex AI Model`资源中*。

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/machine_settings".format(BUCKET_URI)

CPU_LIMIT = "8"  # vCPUs
MEMORY_LIMIT = "8G"


@dsl.pipeline(
    name="component-level-set-resources",
    description="A simple pipeline that requests component-level machine resource",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(epochs: int, model_dir: str, project: str = PROJECT_ID):
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.model import ModelUploadOp
    from kfp.dsl import importer_node

    training_job_task = (
        self_contained_training_component(epochs=epochs, model_dir=model_dir)
        .set_display_name("self-contained-training")
        .set_cpu_limit(CPU_LIMIT)
        .set_memory_limit(MEMORY_LIMIT)
        .add_node_selector_constraint("NVIDIA_TESLA_K80")
        .set_gpu_limit(TRAIN_NGPU)
    )

    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=training_job_task.output,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": DEPLOY_IMAGE,
            },
        },
    ).after(training_job_task)

    _ = ModelUploadOp(
        project=project,
        display_name="mnist_model",
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    ).after(import_unmanaged_model_task)

### 编译并执行管道

接下来，您编译管道然后执行它。管道接受以下参数，这些参数作为字典`parameter_values`传递：

- `model_dir`：保存模型工件的 Cloud Storage 位置。
- `epochs`：训练模型的时代数量。
- `project`：您的项目 ID。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="component_level_settings.yaml",
)

pipeline = aiplatform.PipelineJob(
    display_name="component-level-settings",
    template_path="component_level_settings.yaml",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"model_dir": BUCKET_URI, "epochs": 20, "project": PROJECT_ID},
    enable_caching=False,
)

pipeline.run()

! rm -rf component_level_settings.yaml

查看管道结果

一旦管道完成，您可以查看每个组件步骤的工件输出。

In [ ]:
PROJECT_NUMBER = pipeline.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVAL_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            return EXECUTE_OUTPUT
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            return GCP_RESOURCES
        elif tf.io.gfile.exists(EVAL_METRICS):
            ! gsutil cat $EVAL_METRICS
            return EVAL_METRICS

    return None


print("self-contained-training")
artifacts = print_pipeline_output(pipeline, "self-contained-training")
print("\n\n")
print("importer")
artifacts = print_pipeline_output(pipeline, "importer")
print("\n\n")
print("model-upload")
artifacts = print_pipeline_output(pipeline, "model-upload")
output = !gsutil cat $artifacts
output = json.loads(output[0])
model_id = output["artifacts"]["model"]["artifacts"][0]["metadata"]["resourceName"]
print("\n")
print("MODEL ID", model_id)
print("\n\n")

删除管道任务

在管道任务完成后，您可以使用`delete（）`方法删除管道任务。在完成之前，可以使用`cancel（）`方法取消管道任务。

In [ ]:
pipeline.delete()

### 删除模型

您可以使用`delete()`方法删除由管道生成的`Model`资源。

In [ ]:
model = aiplatform.Model(model_id)
model.delete()

将自包含培训组件转换为`Vertex AI CustomJob`。

接下来，您可以使用实用程序`create_custom_training_job_from_component()`将其转换为`Vertex AI CustomJob`。 这将带来以下好处：

- 将额外的 ML Metadata 跟踪添加为自定义作业。
- 可以设置特定于自定义作业的资源控制。
    - `machine_type`：`CustomJob`的机器（VM）实例。
    - `accelerator_type`：GPU 或 TPU 的类型（如果有）。
    - `accelerator_count`：HW 加速器（GPU/TPU）的数量或零。
    - `replica_count`：作业的 VM 实例数（默认为 1）。
    - `boot_disk_type`：引导磁盘的类型（默认为“pd-ssd”）。
    - `boot_disk_size_gb`：引导磁盘的大小（单位为 GB，默认为 100GB）。

In [ ]:
custom_job_op = create_custom_training_job_from_component(
    self_contained_training_component,
    display_name="test-component",
    machine_type=TRAIN_COMPUTE,
    accelerator_type=TRAIN_GPU.name,
    accelerator_count=TRAIN_NGPU,
)

### 创建CustomJob流水线

接下来，您将创建用于训练此组件的流水线，包括以下步骤：

- *训练模型*。对于这个组件，您设置以下自定义作业级别资源：
    - `machine_type`：机器（VM）实例。
    - `accelerator_type`：容器的VM实例的GPU类型。
    - `accelerator_count`：容器的VM实例的GPU数量。
    - `replica_count`：机器（VM）实例数量。
- *将模型工件导入到Model Container工件中*。
- *上传容器工件到`Vertex AI Model`资源中*。

In [ ]:
@dsl.pipeline(
    name="customjob-set-resources",
    description="A simple pipeline that requests customjob-level machine resource",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(
    epochs: int, model_dir: str, project: str = PROJECT_ID, region: str = REGION
):
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.model import ModelUploadOp
    from kfp.dsl import importer_node

    training_job_task = custom_job_op(
        epochs=epochs, model_dir=model_dir, project=project, location=region
    )

    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=training_job_task.outputs["Output"],
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": DEPLOY_IMAGE,
            },
        },
    ).after(training_job_task)

    _ = ModelUploadOp(
        project=project,
        display_name="mnist_model",
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    ).after(import_unmanaged_model_task)

### 编译和执行管道

接下来，您编译管道，然后执行它。管道接受以下参数，这些参数作为字典`parameter_values`传递：

- `model_dir`：保存模型文件的云存储位置。
- `epochs`：训练模型所需的轮数。
- `project`：您的项目ID。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="customjob_level_settings.yaml",
)

pipeline = aiplatform.PipelineJob(
    display_name="customjob-level-settings",
    template_path="customjob_level_settings.yaml",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"model_dir": BUCKET_URI, "epochs": 20, "project": PROJECT_ID},
    enable_caching=False,
)

pipeline.run()

! rm -rf customjob_level_settings.yaml

查看管道结果

一旦管道完成，您可以查看每个组件步骤的工件输出。

In [ ]:
print("self-contained-training-component")
artifacts = print_pipeline_output(pipeline, "self-contained-training-component")
print("\n\n")
print("importer")
artifacts = print_pipeline_output(pipeline, "importer")
print("\n\n")
print("model-upload")
artifacts = print_pipeline_output(pipeline, "model-upload")
output = !gsutil cat $artifacts
output = json.loads(output[0])
model_id = output["artifacts"]["model"]["artifacts"][0]["metadata"]["resourceName"]
print("\n")
print("MODEL ID", model_id)
print("\n\n")

### 删除管道作业

在管道作业完成后，您可以使用`delete()`方法删除管道作业。在完成之前，您可以使用`cancel()`方法取消管道作业。

In [ ]:
pipeline.delete()

### 删除模型

您可以使用`delete()`方法删除由管道生成的`Model`资源。

In [ ]:
model = aiplatform.Model(model_id)
model.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

In [ ]:
import os

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI